# **Week 1 : Linear Algebra and Calculus**

In this assignment, we shall explore the concepts of analytic and numeric computation of gradients. Further, we will have a look at the computation graph which is a central concept to building a neural network. For learning how gradients are computed analytically, refer to the resources provided in this week.

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQWsYeD8ZlYxFBB33qIn7bwQvP-KuvLZXJOoA&usqp=CAU"
 style="float:center;width:50px;height:50px;">

# **Importing Libraries**
Feel free to import any additional libraries required

In [ ]:
# Import all libraries here
import numpy as np
import math

# Setting the seed for reproducible results


# *Problem 1(a)*

In this problem, we shall be exploring the concepts of analytic and numeric computation of gradients for scalar valued functions. 

\begin{equation}
z = w^{T}x + b \\ 
\hat{y} = \sigma(z) = \frac{1}{1+e^{-z}}\\ 
L(\hat{y}, y) = y.log(\hat{y}) + (1-y).log(1-\hat{y})
\end{equation}

For this set of equations, the vector w, and scalars b, y are to be treated as constants. \\

Now, let us find the analytic gradient of the function L with respect to the function x. That is, compute $\frac{\delta L}{\delta x}$. Write the answer obtained as code in the function provided.

Initialise $w$ to a $10 \times 1$ vector sampled over a standard multivariate gaussian distribution, b to a uniform random variable over the interval $(0, 1)$, y to a bernoulli random variable over $\{0, 1\}$

In [ ]:
# Initialisation : 
def bernoulli(p):
    rvs = np.array([])
    if np.random.rand() <= p:
        a=1
        rvs = np.append(rvs,a)
    else:
        a=0
        rvs = np.append(rvs,a)
    return rvs

w1 = np.random.multivariate_normal([0,0,0,0,0,0,0,0,0,0], np.identity(10))
w2 = np.array(w1)
w = np.reshape(w2, (10,1))
b = np.random.uniform(0,1)
y1 = bernoulli(0.5)
y = y1[0]

In [ ]:
def analytic_grad(x) : 

    ### WRITE CODE BELOW ###
    w_transpose=w.transpose()
    z=np.dot(w_transpose, x)+b
    y_cap=1/(1+math.exp(-z))
    deravative=y*y_cap*math.exp(-z)*w-math.exp(-z)*((1-y)/(1-y_cap))*(y_cap*y_cap)*w
    return deravative
    ### WRITE CODE ABOVE ###

w1 = np.random.multivariate_normal([0,0,0,0,0,0,0,0,0,0], np.identity(10))
w2 = np.array(w1)
x = np.reshape(w2, (10,1))
print(analytic_grad(x))

[[-0.06479532]
 [ 0.09140952]
 [-0.03754852]
 [ 0.00714193]
 [-0.02423793]
 [-0.01474312]
 [-0.05028835]
 [ 0.09520192]
 [ 0.07167586]
 [ 0.00635588]]


# *Problem 1(b)*

Now, we compute the numeric gradient for the function L. Refer to [this](https://stackoverflow.com/questions/38854363/is-there-any-standard-way-to-calculate-the-numerical-gradient) stack exchange post to see how numeric gradients are computed

In [ ]:
gradient=np.zeros((10,1))
def function_value(x):
    w_transpose=w.transpose()
    z=np.dot(w_transpose, x)+b
    y_cap=1/(1+math.exp(-z))
    value=y*math.log(y_cap)+(1-y)*math.log(1-y_cap)

    return value

def function(x,i):
    eps = np.finfo(np.float64).eps
    I=np.identity(10)
    I[i,i]=I[i,i]*(1+math.sqrt(eps))
    x1=np.dot(x.transpose(), I)
    value1=function_value(x1.transpose())
    I[i,i]=I[i,i]/(1+math.sqrt(eps))
    I[i,i]=I[i,i]*(1-math.sqrt(eps))
    x2=np.dot(x.transpose(),I)
    value2=function_value(x2.transpose())
    gradient[i][0]=(value1-value2)/(2*math.sqrt(eps)*x[i][0])

    return 

def numeric_grad(x) : 

    ### WRITE CODE BELOW ###
    for i in range(0,10):
      function(x,i)

    return gradient
    ### WRITE CODE ABOVE ###

w1 = np.random.multivariate_normal([0,0,0,0,0,0,0,0,0,0], np.identity(10))
w2 = np.array(w1)
x = np.reshape(w2, (10,1))
print(numeric_grad(x))

[[-0.17257574]
 [ 0.24345994]
 [-0.10000666]
 [ 0.01902178]
 [-0.06455525]
 [-0.0392668 ]
 [-0.13393789]
 [ 0.25356064]
 [ 0.19090135]
 [ 0.01692824]]


# *Problem 2*

Here, we'll be looking at computational graphs, and their calculus, finding gradients of  variables with respect to other variables in the graph.



We'll be looking at nodes of the graph that are operation based, i.e., each operation performed has a node associated with it.

We'll provide you with example implementations for three of the nodes, the *add* node, the *nth power* node and the *sine* node, you'll have to write the classes for all the other nodes.

# **Multi-input nodes**

In [ ]:
import numpy as np

class Add: 
  
  # A class to add multiple variables together
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be added, with each variable name in datatype str.
                               # Scalar addition is taken care of in a separate node, we could have fit into this node but thought it might be easier if it wasn't.
                               # So cases like "a + 1" are to be done separately, and cases like "b + c + d + 4" can be done as "(b + c + d) + 4" or "b + c + (d + 4)", since our scalar addition supports only one variable and one scalar, we'll get to that later
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be added, as scalars.

  def compute_output(self):
    return np.sum(self.lst_values) # This computes the sum of all the variables
  
  def compute_gradients(self):
    return dict(zip(self.lst_names, np.ones(len(self.lst_names)))) # This gives the gradient of the sum wrt to all the input variables, as a dictionary, will be used later

class Multiply: 
  
  #Everything's almost the same as the add class, but this deals with multiplication of more than 1 variables
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be multiplied, with each variable name in datatype str.
                               # Scalar multiplication is taken care of in a separate node
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be multiplied, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    return np.prod(self.lst_values)
  
  def compute_gradients(self):
    product=np.prod(self.lst_values)
    array=np.zeros(len(self.lst_names))
    for i in range(0, len(self.lst_names)):
      array[i]=product/self.lst_values[i]
    return dict(zip(self.lst_names, array))
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)

A= Multiply(["a","b","c"],[1,2,3])
print(A.compute_gradients())

{'a': 6.0, 'b': 3.0, 'c': 2.0}


# **Scalar multiplication/addition nodes**

In [ ]:
class Add_Scalar: 
  
  # A class to add a variable with a scalar
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be added, with each variable name in datatype str.
                               # This list must only have 2 elements, the first should be the string corresponding to the name of the variable, and the second should be a string of the scalar value to be added.
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be added, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    return np.sum(self.lst_values)

  def compute_gradients(self):
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    array=np.zeros(len(self.lst_names))
    array[0]=1
    array[1]=0
    return dict(zip(self.lst_names, array))


class Multiply_Scalar: 
  
  # A class to multiply a variable with a scalar
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be multiplied, with each variable name in datatype str.
                               # This list must only have 2 elements, the first should be the string corresponding to the name of the variable, and the second should be a string of the scalar value to be multiplied.
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be multiplied, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    return np.prod(self.lst_values)

  def compute_gradients(self):
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    array=np.zeros(len(self.lst_names))
    array[0]=self.lst_values[1]
    array[1]=0
    return dict(zip(self.lst_names, array))

A= Multiply_Scalar(["a","4"],[1,4])
print(A.compute_gradients())
A= Add_Scalar(["a","4"],[1,4])
print(A.compute_gradients())


{'a': 4.0, '4': 0.0}
{'a': 1.0, '4': 0.0}


# **Nodes for special functions**

In [ ]:
class Power:

  def __init__(self, name, value, exponent):
    self.name = name # Name of the variable to be exponentiated
    self.value = value # Value of the variable
    self.exponent = exponent # Value of the exponent
  
  def compute_output(self):
    return np.power(self.value, self.exponent)
  
  def compute_gradients(self):
    return {self.name : self.exponent*(np.power(self.value, (self.exponent - 1)))}

class Sine: 

  # A class to apply the sine function on a variable
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    return np.sin(self.value)
  
  def compute_gradients(self):
    return {self.name : np.cos(self.value)}


class Logarithm:

  # A class to compute the logarithm of a value
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    # Write your code here
    return np.log(self.value)

  def compute_gradients(self):
    # Write your code here
    return {self.name : 1/(self.value)}

class Exponential: 

  # A class to compute the exponential of a value
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    # Write your code here
    return np.exp(self.value)

  def compute_gradients(self):
    # Write your code here
    return {self.name : np.exp(self.value)}



Now that we have these classes, let's use them to actually find gradients of complex networks. We finally bring in the idea of a computational graph, which makes it much easier for the gradients to be computed.

This is the image of the example graph that we want you to work with. 





<div>
<img src="https://drive.google.com/uc?id=1VtI1lf85bG8cO1u_8l0D0rqVGhwHQtPK"
 width="500" height="500">
</div>

\begin{equation}
d = 3a \\ 
e = abc \\ 
f = sin(c) \\ 
g = exp(e) \\ 
h = a + d + g + f
\end{equation}

# **Forward Propogation**

In [ ]:
def forward_prop(a, b, c) : 
    '''
    Input : real numbers a, b, c.

    Outputs : A dictionary of the values at each node with keys as the names of nodes
    Grads : A dictionary of the gradients at each edge with keys as a pair of nodes 
    
    e.g. Grads["ce"] = ...
    '''
    Outputs={}
    Grads={}
    Outputs["a"]=a
    Outputs["b"]=b
    Outputs["c"]=c

    A=Multiply_Scalar(["a","3"],[a,3])
    d=A.compute_output()
    grad=A.compute_gradients()
    Outputs["d"]=d
    Grads["ad"]=grad[A.lst_names[0]]

    B=Multiply(["a","b","c"],[a,b,c])
    e=B.compute_output()
    grad=B.compute_gradients()
    Outputs["e"]=e
    Grads["ae"]=grad[B.lst_names[0]]
    Grads["be"]=grad[B.lst_names[1]]
    Grads["ce"]=grad[B.lst_names[2]]

    C=Sine("c",c)
    f=C.compute_output()
    grad=C.compute_gradients()
    Outputs["f"]=f
    Grads["cf"]=grad[C.name]

    D=Exponential("e",e)
    g=D.compute_output()
    grad=D.compute_gradients()
    Outputs["g"]=g
    Grads["eg"]=grad[D.name]

    E=Add(["a","d", "g", "f"], [a, d, g, f])
    h=E.compute_output()
    grad=E.compute_gradients()
    Outputs["h"]=h
    Grads["ah"]=grad[E.lst_names[0]]
    Grads["dh"]=grad[E.lst_names[1]]
    Grads["gh"]=grad[E.lst_names[2]]
    Grads["fh"]=grad[E.lst_names[3]]
    
    return (Outputs, Grads)

print(forward_prop(1,2,3))

({'a': 1, 'b': 2, 'c': 3, 'd': 3, 'e': 6, 'f': 0.1411200080598672, 'g': 403.4287934927351, 'h': 407.569913500795}, {'ad': 3.0, 'ae': 6.0, 'be': 3.0, 'ce': 2.0, 'cf': -0.9899924966004454, 'eg': 403.4287934927351, 'ah': 1.0, 'dh': 1.0, 'gh': 1.0, 'fh': 1.0})


# **Backward Propogation**

Most of the usage of computational graphs in Machine Learning centers around finding the gradients of a particular loss, wrt to all the parameters. Here, your task is to do a simpler version of that.

Use the classes you wrote to calculate the following gradients : 


*   $ \frac{\partial h}{\partial a}$
*   $ \frac{\partial h}{\partial b}$
*   $ \frac{\partial h}{\partial c}$

Use the technique of *backpropogation* to code out the gradients step-wise, along all possible chains of the graph starting from $h$ and ending at $a,b,c$ respectively. 

Don't try to directly get these values without backpropogation, it might be easier here, but with complicated neural networks it wouldn't be :) 



In [ ]:
def backward_prop(a, b, c, Outputs, Grads) : 
    '''
    Input : a, b, c (input layer); Outputs (values at each node); Grads (gradients stored at each edge)
    Retuns : result (gradients w.r.t a, b, c)
    '''
    result=[Grads["ad"]*Grads["dh"], Grads["gh"]*Grads["eg"]*Grads["be"], Grads["cf"]*Grads["fh"]]

    return result

# **Combining both processes**

For the purpose of values, assume that $a = 3, b = 1, c = 2$. Here, we call both forward and backward propogation functions to demonstrate functionality of the functions above.

In [ ]:
# Initialisation
a = 3
b = 1
c = 2

# Forward propogation                                                                                                                                                                                  
(Outputs, Grads) = forward_prop(a, b, c)
print(f'Value obtained upon forward propogation : {Outputs["h"]}')

# Backward propogation
result = backward_prop(a, b, c, Outputs, Grads)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
print(f'Values of gradients are : {result[0]}, {result[1]}, {result[2]}')

Value obtained upon forward propogation : 416.3380909195608
Values of gradients are : 3.0, 2420.5727609564105, -0.4161468365471424


# **Submission Instructions**

Upload this notebook on your github classroom repository by the name Week1.ipynb